# R-NET

In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import xavier_initializer as xinit

## Placeholders

In [2]:
Lp = 100
Lq = 20
B = 8
embed_dim = 150
hdim = 2*embed_dim
vocab_size = 10000

In [3]:
tf.reset_default_graph()
p = tf.placeholder(tf.int32, shape=[None, Lp], name='passage')
q = tf.placeholder(tf.int32, shape=[None, Lq], name='question')

## Embedding

**TODO**
- [ ] Include character embedding for OOV tokens

In [4]:
E = tf.get_variable('E', dtype=tf.float32, shape=[vocab_size, embed_dim], initializer=xinit())
qe = tf.nn.embedding_lookup(E, q)
pe = tf.nn.embedding_lookup(E, p)

## Question Encoder

In [5]:
from lib.recurrence import *

In [6]:
(qstates_f, qstates_b), _ = bi_net(cell_f= gru_n(embed_dim,3), cell_b=gru_n(embed_dim,3),
                                inputs=qe, batch_size=B, timesteps=Lq,
                                scope='q_enc')
qstates = tf.concat([qstates_f, qstates_b], axis=-1)

## Passage Encoder

In [7]:
(pstates_f, pstates_b), _ = bi_net(cell_f= gru_n(embed_dim,3), cell_b=gru_n(embed_dim,3),
                                inputs=pe, batch_size=B, timesteps=Lp,
                                scope='a_enc')
pstates = tf.concat([pstates_f, pstates_b], axis=-1)

### Question-aware Passage Representation

In [8]:
with tf.variable_scope('qp'):
    qp_states = gated_attention_net(qstates, pstates,# encoded representation of text
                                    tf.zeros(dtype=tf.float32, shape=[B,hdim]), # notice d*2
                                    batch_size=B, d=hdim, La=Lq, Lb=Lp,
                                    scope='qp')                                    

## Self-matching Representation

In [9]:
with tf.variable_scope('pp'):
    # convert qp_states to time-major
    qp_states = tf.transpose(qp_states, [1,0,-1])
    pp_states = gated_attention_net(pstates, qp_states,# encoded representation of text
                                    tf.zeros(dtype=tf.float32, shape=[B,hdim]), # notice d*2
                                    batch_size=B, d=hdim, La=Lp, Lb=Lp,
                                    scope='pp')                                    